# Concurrency with asyncio

## Compare vs Threading

In [1]:
# spinner_thread.py

# credits: Adapted from Michele Simionato's
# multiprocessing example in the python-list:
# https://mail.python.org/pipermail/python-list/2009-February/538048.html

# BEGIN SPINNER_THREAD
import threading
import itertools
import time
import sys


def spin(msg, done):  # <2>
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):  # <3>
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))  # <4>
        if done.wait(.1):  # <5>
            break
    write(' ' * len(status) + '\x08' * len(status))  # <6>


def slow_function():  # <7>
    # pretend waiting a long time for I/O
    time.sleep(3)  # <8>
    return 42


def supervisor():  # <9>
    done = threading.Event()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', done))
    print('spinner object:', spinner)  # <10>
    spinner.start()  # <11>
    result = slow_function()  # <12>
    done.set()  # <13>
    spinner.join()  # <14>
    return result


def main():
    result = supervisor()  # <15>
    print('Answer:', result)


if __name__ == '__main__':
    main()
# END SPINNER_THREAD

spinner object: <Thread(Thread-4, initial)>
| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!/ thinking!- thinking!\ thinking!| thinking!           Answer: 42


## And now with an Asyncio coroutine

In [4]:
import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/=\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)
            # using the combo of yield from and asyncio.sleep
            # we can sleep without blocking the event loop
        except asyncio.CancelledError: 
            break
    write(' ' * len(status) + '\x08' * len(status))

@asyncio.coroutine
def slow_function():
    # pretend waiting a long time for I/O
    yield from asyncio.sleep(3) #  handles control flow to the main loop,
    # and wil resume the coroutine after the sleep delay
    return 42

@asyncio.coroutine
def supervisor():
    # asyncio.async schedules the the spin coro to run
    # wrapping it in a Task object, which is returned immediatly
    spinner = asyncio.async(spin('thinking!'))  
    print('spinner object: ', spinner) # display the task object
    result = yield from slow_function() # drive the slow function
    # when that is done, we get the returned value
    # meanwhile the event loop will continue running because slow_function
    # ulimately uses yield from asyncio.sleep(3) to hand control
    # back to the main loop
    spinner.cancel() # ends the coro
    return result

def main():
    loop = asyncio.get_event_loop()  # get a ref to the event loop
    result = loop.run_until_complete(supervisor())  # drive the supervisor
    # to completion. The return value of the coor is the return value
    # of this call
    print('Answer: ', result)

if __name__ == '__main__':
    main()

spinner object:  <Task pending coro=<spin() running at <ipython-input-4-f05f121b810c>:5>>
| thinking!/ thinking!= thinking!\ thinking!| thinking!/ thinking!= thinking!\ thinking!| thinking!/ thinking!= thinking!\ thinking!| thinking!/ thinking!= thinking!\ thinking!| thinking!/ thinking!= thinking!\ thinking!| thinking!/ thinking!= thinking!\ thinking!| thinking!/ thinking!= thinking!\ thinking!| thinking!           Answer:  42


Note:  Never use time.sleep(...) in asyncio coro's unless you want
to block the main thread, therefore freezing the event loop 
and prob whole application.... 

If coro must spend time doing nothing, use: yield from asyncio.sleep(DELAY)


In [ ]:
# quick coro experiment code
import asyncio
def run_sync(coro_or_future):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(coro_or_future)

a = run_sync(some_coroutine())

## Downloading with asyncio and aiohttp
asyncio supports TCP and UDP directly.

For HTTP or any other protocol, we need 3rd party packages.

aiohttp is the one to use or asyncio HTTP clients right now.

Idea for below example:
- Single threaded program where a main-loop activates queued coroutines one by one
- Each coroutine advances a few steps, then yields control back to the main loop, which activates the next coroutine in the queue.